In [34]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import scipy.stats
from scipy.stats import uniform

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix

from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

In [35]:
df = pd.read_csv('D:\File_Analisa_Python\ML_ProbabilityDefault.csv', sep='|')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2072516 entries, 0 to 2072515
Data columns (total 29 columns):
 #   Column               Dtype  
---  ------               -----  
 0   POSISI               object 
 1   ROA                  float64
 2   CBAL                 float64
 3   GAJI_TOTAL           float64
 4   JML_PENGHASILAN      float64
 5   ORGAMT               float64
 6   JUMLAH_CICILAN       float64
 7   TUNGGAKAN_DENDA      float64
 8   TUNGGAKAN_POKOK      float64
 9   TUNGGAKAN_BUNGA      float64
 10  SUKU_BUNGA           float64
 11  JANGKA_WAKTU_KREDIT  float64
 12  MARITAL_ID           object 
 13  GENDER_ID            float64
 14  KOL                  float64
 15  JENIS                float64
 16  LONDSC               object 
 17  AGUNAN               object 
 18  NILAI_AGUNAN         float64
 19  LUAS_BGN_AGUNAN      float64
 20  LUAS_TNH_BANGUNAN    float64
 21  KET_AGF              float64
 22  SEGMEN               float64
 23  RASIO                float64
 24

In [37]:
df = df.drop(index=2072515)
df.isna().sum()

POSISI                 0
ROA                    0
CBAL                   0
GAJI_TOTAL             0
JML_PENGHASILAN        0
ORGAMT                 0
JUMLAH_CICILAN         0
TUNGGAKAN_DENDA        0
TUNGGAKAN_POKOK        0
TUNGGAKAN_BUNGA        0
SUKU_BUNGA             0
JANGKA_WAKTU_KREDIT    0
MARITAL_ID             0
GENDER_ID              0
KOL                    1
JENIS                  0
LONDSC                 0
AGUNAN                 0
NILAI_AGUNAN           0
LUAS_BGN_AGUNAN        0
LUAS_TNH_BANGUNAN      0
KET_AGF                0
SEGMEN                 0
RASIO                  0
STATREK                0
TARGET                 0
TAHUN                  0
BULAN                  0
BLN                    0
dtype: int64

In [43]:
df['KOL'] = df['KOL'].fillna(0)

df.isna().sum()

POSISI                 0
ROA                    0
CBAL                   0
GAJI_TOTAL             0
JML_PENGHASILAN        0
ORGAMT                 0
JUMLAH_CICILAN         0
TUNGGAKAN_DENDA        0
TUNGGAKAN_POKOK        0
TUNGGAKAN_BUNGA        0
SUKU_BUNGA             0
JANGKA_WAKTU_KREDIT    0
MARITAL_ID             0
GENDER_ID              0
KOL                    0
JENIS                  0
LONDSC                 0
AGUNAN                 0
NILAI_AGUNAN           0
LUAS_BGN_AGUNAN        0
LUAS_TNH_BANGUNAN      0
KET_AGF                0
SEGMEN                 0
RASIO                  0
STATREK                0
TARGET                 0
TAHUN                  0
BULAN                  0
BLN                    0
dtype: int64

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2072515 entries, 0 to 2072514
Data columns (total 29 columns):
 #   Column               Dtype  
---  ------               -----  
 0   POSISI               object 
 1   ROA                  float64
 2   CBAL                 float64
 3   GAJI_TOTAL           float64
 4   JML_PENGHASILAN      float64
 5   ORGAMT               float64
 6   JUMLAH_CICILAN       float64
 7   TUNGGAKAN_DENDA      float64
 8   TUNGGAKAN_POKOK      float64
 9   TUNGGAKAN_BUNGA      float64
 10  SUKU_BUNGA           float64
 11  JANGKA_WAKTU_KREDIT  float64
 12  MARITAL_ID           object 
 13  GENDER_ID            float64
 14  KOL                  float64
 15  JENIS                float64
 16  LONDSC               object 
 17  AGUNAN               object 
 18  NILAI_AGUNAN         float64
 19  LUAS_BGN_AGUNAN      float64
 20  LUAS_TNH_BANGUNAN    float64
 21  KET_AGF              float64
 22  SEGMEN               float64
 23  RASIO                float64
 24

# Encoding Label

In [52]:
df[['MARITAL_ID','LONDSC']].groupby('MARITAL_ID').count()

,LONDSC
MARITAL_ID,
A,1570872
B,467813
C,17490
D,6267
E,2114
F,974
G,273
Z,6712
